# Clasificar proyectos CORFO según tipo de enfermedad

El objetivo de este notebook es clasificar cada proyecto CORFO de acuerdo a la enfermedad abordada en el proyecto: `cáncer`, `diabetes`, `cardiaca` (para enfermedades cardiovasculares) u `otro`, si es que no trata ninguna de las enfermedades señaladas.

In [ ]:
!pip install openai
!pip install langchain
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import numpy as np
import pandas as pd
from csv import writer
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata

## Cargar datos

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Datos"
save_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT4_Análisis temática definida/Proyectos/CORFO/Resultados"
df = pd.read_excel(os.path.join(data_dir, "DataInnovacion - InnovaChile Corfo.xlsx"),  skiprows=1)
df

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
0,23SN-242063,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77003755-7,Bebidas Saludables SpA,Mejorar las capacidades de las bebidas Masala ...,...,2019-05-20 00:00:00,Turismo,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,Sí,NaN,NaN,Sin tendencia
1,23SN-242058,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,76228136-8,Tecagromed SpA,TECFILMS: Desarrollo de un biomaterial en form...,...,2012-11-20 00:00:00,Comercio y retail,Intercambiar,Economía Circular,R2 Reducir,Uso y fabricación de productos más inteligentes,No,NaN,NaN,Química Verde (Green Chemestry)
2,23SN-242042,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77618332-6,Green Focus Solutions SpA,Kit molecular para la identificación rápida de...,...,2022-12-08 00:00:00,Biotecnológico,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Nanotecnología
3,23SN-242041,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77704466-4,Zeroglut SpA,Evaluación de tecnologías de envasado para inc...,...,2023-02-21 00:00:00,Alimentos (excepto vitivinícola),Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
4,23SN-242036,Renuévate,Subsidio,Súmate a Innovar,Súmate a Innovar,VIGENTE,Persona Jurídica constituida en Chile,77147033-5,La mar spa,Desarrollo de un pastel de jaibas listo para c...,...,2020-04-23 00:00:00,Comercio y retail,Remanufacturar,Economía Circular,R6 Remanufacturar,Extensión de vida útil del producto y sus partes,No,NaN,NaN,Sin tendencia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9439,09ANT-7178,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,99576630-2,OXZO SOCIEDAD ANONIMA,introducción al mercado nacional de plataforma...,...,2004-12-01 00:00:00,Manufactura de no metálicos,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9440,09ANT-7131,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,77183150-8,METALCOP INGENIERIA LIMITADA,nuevo sistema apertura tapa balde para palas d...,...,1998-07-10 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9441,09ACS-7457,Desarrolla innovación,Subsidio,Apoyo Elaboración Capital Semilla,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,96691330-4,DICTUC S.A.,apoyo a la elaboración de estudios de preinver...,...,1994-03-02 00:00:00,Servicios de ingeniería o de conocimiento,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9442,09AAP-6741,Renuévate,Subsidio,Apoyo A La Atracción De Profesionales Y Técnicos,Capital Humano,FINALIZADO,Persona Jurídica constituida en Chile,76832220-1,ORION FORESTAL SA,desarrollo de sistemas de recolección de bioma...,...,2007-05-11 00:00:00,Forestal,Intercambiar,Economía Circular,NaN,Uso y fabricación de productos más inteligentes,No,NaN,Biocombustible,Sin tendencia


## Preprocesar datos

In [ ]:
df[df["Objetivo"].isna()]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
2876,18ITE2-102868,Desarrolla innovación,Subsidio,Línea 2: Validación Y Empaquetamiento De Innov...,Validación y empaquetamiento de innovaciones,FINALIZADO,Persona Jurídica constituida en Chile,76276688-4,SOCIEDAD VÖLKER & AQUEA LIMITADA,estandarización de la producción de australcel...,...,2013-07-04 00:00:00,Agrícola (excepto vitivinícola),NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Sin tendencia
2916,18EUREID-105748,Desarrolla innovación con I+D,Subsidio,LÍNEA 1: I+D APLICADA EN EMPRESAS-CORFO EUREKA,I+D Aplicada en Empresas,FINALIZADO,Persona Jurídica constituida en Chile,76063790-4,INGENIERIA REDCO LIMITADA,sistema predictivo multi-fabricante de mantenc...,...,2004-03-01 00:00:00,Tecnologías de la información,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
5103,16PIUK-57404,Desarrolla innovación,Subsidio,Prototipos De Innovacion Corfo - Innova Uk,Prototipos de innovación,FINALIZADO,Persona Jurídica constituida en Chile,77762940-9,ANGLO AMERICAN SUR S.A.,manejo del agua y la energia para el desarroll...,...,2002-06-07 00:00:00,Minería y metalurgia extractiva,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia
5539,16CET2-63914,Entorno para la innovación,Subsidio,Centros De Extensionismo Tecnológico,Fortalece Pyme,FINALIZADO,Persona Jurídica constituida en Chile,70885500-6,UNIVERSIDAD DE TALCA,cetlog: centro de extensionismo logístico,...,1993-01-01 00:00:00,Educación,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7389,12SSAF-17789,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,81668700-4,UNIVERSIDAD TÉCNICA FEDERICO SANTA MARÍA,boosterup: programa de aceleración estrategica...,...,1993-01-01 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7391,12SSAF-14855,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,76109353-3,UDD VENTURES S.A.,fondo ssaf udd ventures,...,2010-08-20 00:00:00,Servicios de ingeniería o de conocimiento,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
7553,12PAE-16027,Entorno para la innovación,Subsidio,Apoyo Al Entorno Emprendedor,Apoyo al entorno del emprendimiento,FINALIZADO,Persona Jurídica constituida en Chile,96863740-1,FUNDES CHILE SPA,programa de emprendimiento femenino mundo mujer,...,1998-06-25 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
8066,11SUP-9599,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,babelverse,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
8229,11SUP-11576,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,patagonia sostenible,...,NaN,NaN,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Sin tendencia
8299,11SSAF-9928,Desarrolla innovación,Subsidio,Subsidio Semilla De Asignación Flexible Para E...,Capital Semilla,FINALIZADO,Persona Jurídica constituida en Chile,70300000-2,FUNDACIÓN CHILE,fondo ssaf fundación chile,...,1993-01-01 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia


In [ ]:
df["Objetivo"].value_counts()

Información no pública                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   1153
.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         108
sin informac

22 filas sin objetivo.

In [ ]:
df[df["Título"].isna()]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Inicio Actividad Económica,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final
9014,10NTEC-7373,Entorno para la innovación,Subsidio,Línea del Apoyo al Extensionismo,Nodos,FINALIZADO,Persona Jurídica constituida en Chile,65386530-9,ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO ES...,NaN,...,2004-07-06 00:00:00,Asociaciones y organizaciones no empresariales...,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia
9266,10ANT-8103,Consolida y Expande,Subsidio,Apoyo Negocios Tecnológicos,Apoyo Negocios Tecnológicos,FINALIZADO,Persona Jurídica constituida en Chile,96825800-1,COMERCIAL Y DISTRIBUIDORA BERTONATI S.A.,NaN,...,1997-07-23 00:00:00,Comercio y retail,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia


22 filas sin título.

In [ ]:
def merge_title_and_objectives(title, objective):
  if type(title) != str and type(objective) != str:
    return None
  if type(objective) != str or objective.lower().strip() in ["información no pública", ".", "sin información"]:
    return f"TÍTULO: {title}"
  return f"TÍTULO: {title}. OBJETIVO: {objective}"

df["glosa"] = df.apply(lambda row: merge_title_and_objectives(row["Título"], row["Objetivo"]), axis=1)

In [ ]:
df["glosa"]

0       TÍTULO: Mejorar las capacidades de las bebidas...
1       TÍTULO: TECFILMS: Desarrollo de un biomaterial...
2       TÍTULO: Kit molecular para la identificación r...
3       TÍTULO: Evaluación de tecnologías de envasado ...
4       TÍTULO: Desarrollo de un pastel de jaibas list...
                              ...                        
9439    TÍTULO: introducción al mercado nacional de pl...
9440    TÍTULO: nuevo sistema apertura tapa balde para...
9441    TÍTULO: apoyo a la elaboración de estudios de ...
9442    TÍTULO: desarrollo de sistemas de recolección ...
9443    TÍTULO: biomasa forestal del manejo sustentabl...
Name: glosa, Length: 9444, dtype: object

In [ ]:
df_valid = df[~df["glosa"].isna()]
len(df_valid)

9442

In [ ]:
df_valid[3370:3380]

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final,glosa
3370,18IS9-98252,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,75999510-4,CIUDAD VIVA,rutas bakanes para conectar escuelas rurales-u...,...,Servicios empresariales administrativos y de a...,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: rutas bakanes para conectar escuelas r...
3371,18IS9-98246,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,elaboración por mujeres campesinas de producto...,...,NaN,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: elaboración por mujeres campesinas de ...
3372,18IS9-98241,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,87912900-1,UNIVERSIDAD DE LA FRONTERA,filtro ambiental y socialmente sustentable par...,...,Educación,NaN,Producción Sostenible,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: filtro ambiental y socialmente sustent...
3373,18IS9-98225,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76316644-9,SOCIEDAD DE APRENDIZAJE Y ENTRETENCIÓN HIFIVE ...,plataforma multicultural bilingüe para niños,...,Comercio y retail,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: plataforma multicultural bilingüe para...
3374,18IS9-98096,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,61312000-9,INSTITUTO DE INVESTIGACIONES AGROPECUARIAS - INIA,modelo participativo de valorización productiv...,...,Servicios de ingeniería o de conocimiento,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: modelo participativo de valorización p...
3375,18IS9-97652,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,70777500-9,UNIVERSIDAD ARTURO PRAT- UNAP,rcicla2: programa de formación ambiental juvenil,...,Educación,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: rcicla2: programa de formación ambient...
3376,18IS6-98635,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,65092287-5,FUNDACIÓN CENTRO DE ESTUDIOS DE MONTAÑA,senderismo y escalada para la superación,...,Asociaciones y organizaciones no empresariales...,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: senderismo y escalada para la superaci...
3377,18IS6-98632,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76467250-K,INGLOBO CHILE ASESORIAS TECNICAS EN CONTROL DE...,agroinm integremos a inmigrantes en el agro,...,Servicios de ingeniería o de conocimiento,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: agroinm integremos a inmigrantes en el...
3378,18IS6-98630,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76882776-1,FRUTA SOLAR Y ECOTECNOLOGÍAS,modelo de gestión participativa para la transi...,...,Servicios de ingeniería o de conocimiento,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: modelo de gestión participativa para l...
3379,18IS6-98625,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76667973-0,7 PÉTA

Se sacaron dos filas donde no había ni título ni objetivo.

## Clasificación

Inicializar modelo

In [ ]:
api_key = userdata.get('openai_api_key')
chat = ChatOpenAI(model_name="gpt-4",
                    openai_api_key=api_key,
                    temperature=0,
                    max_tokens=3500)
chat

Le daremos al LLM una lista de instrucciones, pidiéndole clasificar cada proyecto en `cáncer`, `diabetes`, `cardiaca` u `otro`. Luego, le pasaremos una lista de proyectos para que clasifique. Iteraremos en bloques de 10 proyectos, e iremos guardando de inmediato los resultados.

In [ ]:
system_prompt = ("Eres un experto en evaluación de proyectos de investigación. " +
                 "Recibiras una lista de proyectos, cada uno empezando con un guión (-). " +
                 "Algunos proyectos tienen un título y un objetivo, mientras otros sólo tienen título. "
                 "Tu trabajo es determinar para cada proyecto, si habla de cáncer, de diabetes, de enfermedades cardiacas u otro tema. " +
                 "Si el tema del proyecto es cáncer, debes responder 'cáncer'. " +
                 "Si el tema del proyecto es diabetes, debes responder 'diabetes'. " +
                 "Si el tema del proyecto es enfermedades cardiacas, debes responder 'cardiaca'. " +
                 "Si el tema del proyecto es algo distinto, debes responder 'otro.'. " +
                 "No pongas guiones al principio de cada respuesta. Sólo debes responder la clasificación correcta en cada línea.")

system_message_prompt = SystemMessage(content=system_prompt)
n_projects_prompt = 10

filepath = os.path.join(save_dir, "clasificaciones_corfo_enfermedades_especificas.csv")
if not os.path.exists(filepath):
  with open(filepath, "w") as fp:
    print("Creando archivo...")
    csv_writer = writer(fp)
    header = ["código", "glosa", "clasificación"]
    csv_writer.writerow(header)


for iii in tqdm(range(3380, len(df_valid), n_projects_prompt)):
  current_df = df_valid.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["glosa"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["Código Proyecto"].values, current_df["glosa"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

  0%|          | 1/607 [00:04<42:06,  4.17s/it]

Distinto número de respuestas que de proyectos
3380 3381


  1%|          | 6/607 [00:25<42:23,  4.23s/it]

Distinto número de respuestas que de proyectos
3430 3431


 19%|█▉        | 117/607 [07:04<30:47,  3.77s/it]

...

 62%|██████▏   | 379/607 [22:33<13:33,  3.57s/it]

Distinto número de respuestas que de proyectos
7160 7161


 91%|█████████ | 551/607 [32:37<03:32,  3.80s/it]

Distinto número de respuestas que de proyectos
8880 8881


100%|██████████| 607/607 [35:45<00:00,  3.53s/it]


### Datos faltantes

Vemos que hubo algunas iteraciones que fallaron, así que intentaremos de nuevo procesar esos datos.

In [ ]:
df_missing = df_valid.iloc[np.r_[3380:3390, 3430:3440, 4540:4550, 6150:6160, 6680:6690, 6760:6770, 7160:7170, 8880:8890]]
df_missing

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final,glosa
3380,18IS6-98611,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76604340-2,BC-TIC SPA,prototipo reciclaje inclusivo y colaborativo,...,Tecnologías de la información,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: prototipo reciclaje inclusivo y colabo...
3381,18IS6-98607,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76487289-4,"ASESORIA, INVESTIGACIÓN Y PRODUCCIÓN DE ALIMEN...",sistema de reciclaje de frutas para convertirl...,...,Servicios de ingeniería o de conocimiento,Simbiosis Industrial,Economía Circular,R8 Reciclar,Aplicación útil de materiales,No,NaN,NaN,Sin tendencia,TÍTULO: sistema de reciclaje de frutas para co...
3382,18IS6-98575,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76363037-4,CREACAP CAPACITACIONES SPA,escuela de cocina territorial y sustentable,...,Servicios de ingeniería o de conocimiento,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: escuela de cocina territorial y susten...
3383,18IS6-98557,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,65041318-0,FUNDACIÓN GRANDES VALORES,"""el partido de mi barrio rancagua"", la comunid...",...,Asociaciones y organizaciones no empresariales...,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,"TÍTULO: ""el partido de mi barrio rancagua"", la..."
3384,18IS6-98389,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,modelo de mediación ambiental y certificación ...,...,NaN,NaN,Consultoría o estudios para la sostenibilidad,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: modelo de mediación ambiental y certif...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8885,10SUP-8780,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,h2020,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: h2020. OBJETIVO: it is estimated that ...
8886,10SUP-8779,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,knowledgestreem,...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: knowledgestreem. OBJETIVO: the objecti...
8887,10SUP-8778,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,androidim (code name),...,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: androidim (code name). OBJETIVO: who a...
8888,10SUP-8763,Desarrolla innovación,Subsidio,Startup (I),Startup,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,helio-green,...,NaN,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: helio-green. OBJETIVO: overview we ha...


Una hipótesis inicial es que el proceso falló en esas iteraciones porque el prompt era muy largo. Así que en esta primera pasada, dejaremos afuera los proyectos con glosa mayor a 1000 caracteres.

In [ ]:
df_missing_short = df_missing[df_missing["glosa"].str.len() < 1000]
df_missing_short

,Código Proyecto,Foco Apoyo,Tipo Intervención,Instrumento,Instrumento Homologado,Estado Data,Tipo Persona,Rut Beneficiario,Beneficiario,Título,...,Sector Económico,Patron principal asociado,Tipo proyecto,R principal,Estrategia R Principal,Ley REP,Ley REP (Sí/No),ERNC,Tendencia Final,glosa
3380,18IS6-98611,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76604340-2,BC-TIC SPA,prototipo reciclaje inclusivo y colaborativo,...,Tecnologías de la información,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: prototipo reciclaje inclusivo y colabo...
3381,18IS6-98607,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76487289-4,"ASESORIA, INVESTIGACIÓN Y PRODUCCIÓN DE ALIMEN...",sistema de reciclaje de frutas para convertirl...,...,Servicios de ingeniería o de conocimiento,Simbiosis Industrial,Economía Circular,R8 Reciclar,Aplicación útil de materiales,No,NaN,NaN,Sin tendencia,TÍTULO: sistema de reciclaje de frutas para co...
3383,18IS6-98557,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,65041318-0,FUNDACIÓN GRANDES VALORES,"""el partido de mi barrio rancagua"", la comunid...",...,Asociaciones y organizaciones no empresariales...,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,"TÍTULO: ""el partido de mi barrio rancagua"", la..."
3384,18IS6-98389,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Natural,Persona Natural,Persona Natural,modelo de mediación ambiental y certificación ...,...,NaN,NaN,Consultoría o estudios para la sostenibilidad,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: modelo de mediación ambiental y certif...
3385,18IS6-97513,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76768374-K,QUIMÍCA Y TECNOLOGÍA MAIT LIMITADA,cuidemos nuestro aire,...,Servicios empresariales administrativos y de a...,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: cuidemos nuestro aire. OBJETIVO: media...
3386,18IS15-98171,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76653096-6,EDUCA E INNOVA E.I.R.L.,reciclando aprendo,...,Banca y sector financiero,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: reciclando aprendo. OBJETIVO: aplicar ...
3387,18IS15-98149,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76884200-0,GLORIA OCHOA SERVICIOS PROFESIONALES,juego ¿quién soy?,...,Servicios de ingeniería o de conocimiento,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: juego ¿quién soy?. OBJETIVO: desarroll...
3388,18IS15-98111,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,76388269-1,CARLOS ALBERTO ARACENA ESCUDERO INGENIERÍA AMB...,canal digital para artes y oficios de població...,...,NaN,NaN,Innovación Social,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: canal digital para artes y oficios de ...
3389,18IS15-98110,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constituida en Chile,65162143-7,"""ASOCIACIÓN DE DESARROLLO AMBIENTAL Y SOCIAL, ...",escuela de recicladores de base,...,Asociaciones y organizaciones no empresariales...,NaN,Habilitador para la EC,NaN,NaN,No,NaN,NaN,Sin tendencia,TÍTULO: escuela de recicladores de base. OBJET...
3430,18IS1-97611,Desarrolla innovación,Subsidio,Prototipos De Innovacion Social,Prototipos De Innovacion Social,FINALIZADO,Persona Jurídica constitui

In [ ]:
for iii in tqdm(range(0, len(df_missing_short), n_projects_prompt)):
  current_df = df_missing_short.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["glosa"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["Código Proyecto"].values, current_df["glosa"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

100%|██████████| 5/5 [00:20<00:00,  4.06s/it]


Para el resto de los proyectos, iteraremos en grupos de 2 proyectos.

In [ ]:
df_missing_long = df_missing[df_missing["glosa"].str.len() >= 1000]
n_projects_prompt = 2

for iii in tqdm(range(0, len(df_missing_long), n_projects_prompt)):
  current_df = df_missing_long.iloc[iii: iii + n_projects_prompt]
  projects = "\n".join(("- " + current_df["glosa"]).values)
  messages = [
    system_message_prompt, HumanMessage(content=projects)
  ]
  ai_response = chat(messages)
  responses = ai_response.content.split("\n")
  if len(current_df) != len(responses):
    print(f"Distinto número de respuestas que de proyectos")
    print(iii, iii + 10)
  else:
    rows = list(zip(current_df["Código Proyecto"].values, current_df["glosa"], responses))
    with open(filepath, "a") as fp:
      csv_writer = writer(fp)
      csv_writer.writerows(rows)

100%|██████████| 16/16 [00:20<00:00,  1.30s/it]


## Revisar clasificaciones

In [ ]:
df_class = pd.read_csv(filepath)
df_class

,código,glosa,clasificación
0,23SN-242063,TÍTULO: Mejorar las capacidades de las bebidas...,otro
1,23SN-242058,TÍTULO: TECFILMS: Desarrollo de un biomaterial...,otro
2,23SN-242042,TÍTULO: Kit molecular para la identificación r...,otro
3,23SN-242041,TÍTULO: Evaluación de tecnologías de envasado ...,otro
4,23SN-242036,TÍTULO: Desarrollo de un pastel de jaibas list...,otro
...,...,...,...
9437,10SUP-8805,TÍTULO: entrustet. OBJETIVO: entrustet is a fr...,otro
9438,10SUP-8800,TÍTULO: assertid - online identity verificatio...,otro
9439,10SUP-8780,TÍTULO: h2020. OBJETIVO: it is estimated that ...,otro
9440,10SUP-8778,TÍTULO: androidim (code name). OBJETIVO: who a...,otro


In [ ]:
ids_1 = set(df_valid["Código Proyecto"].unique())
ids_2 = set(df_class["código"].unique())

ids_1 == ids_2

True

Todos los proyectos tienen clasificación :)

In [ ]:
df_class["clasificación"].value_counts()

otro                8436
otro.                787
- otro                74
cáncer                72
diabetes              33
cardiaca              31
- cáncer               5
diabetes.              1
- diabetes             1
diabetes, cáncer       1
cáncer.                1
Name: clasificación, dtype: int64

In [ ]:
df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()

<ipython-input-47-39a0d2e375fc>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_class["clasificación_normalizada"] = df_class["clasificación"].str.replace(".", "").str.replace("-", "").str.strip()


In [ ]:
df_class["clasificación_normalizada"].value_counts()

otro                9297
cáncer                78
diabetes              35
cardiaca              31
diabetes, cáncer       1
Name: clasificación_normalizada, dtype: int64

In [ ]:
df_class.to_csv(os.path.join(save_dir, "clasificaciones_corfo_enfermedades_especificas_limpia.csv"), index=False)